In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

c:\Programming\python\kt_aivle\webtoon\myvenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def find_sim(story1, story2, checkpoints):
    
    story_vector1 = checkpoints.encode(story1)
    story_vector2 = checkpoints.encode(story2)
    story_sims = util.cos_sim(story_vector1,story_vector2)
    return story_sims

In [7]:
import pymongo

conn = pymongo.MongoClient()
webtoon_db = conn.webtoon_db
webtoon_collection = webtoon_db.webtoon_collection

In [20]:
docs = webtoon_collection.find({},{'story':1,'title':1})
checkpoints = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
story_sim_list = []
for base in docs:
    for compare in docs:
        story_sim_list.append((find_sim(base['story'],compare['story'],checkpoints).tolist()[0], compare['title']))
    print(story_sim_list)


[(tensor([[0.4095]]), '퀘스트지상주의'), (tensor([[0.3216]]), '장씨세가 호위무사'), (tensor([[0.3388]]), '참교육'), (tensor([[0.2527]]), '윈드브레이커'), (tensor([[0.3649]]), '팔이피플'), (tensor([[0.2770]]), '신화급 귀속 아이템을 손에 넣었다'), (tensor([[0.1646]]), '앵무살수'), (tensor([[0.2335]]), '소녀의 세계'), (tensor([[0.2720]]), '버림받은 왕녀의 은밀한 침실'), (tensor([[0.3186]]), '똑 닮은 딸'), (tensor([[0.1912]]), '호랑신랑뎐'), (tensor([[0.2087]]), '퍼니게임'), (tensor([[0.2407]]), '잔불의 기사'), (tensor([[0.3198]]), '신군'), (tensor([[0.2786]]), '히어로메이커'), (tensor([[0.1834]]), '물어보는 사이'), (tensor([[0.2070]]), '북부 공작님을 유혹하겠습니다'), (tensor([[0.1689]]), '원하나'), (tensor([[0.0743]]), '꼬리잡기'), (tensor([[0.3612]]), '순정말고 순종'), (tensor([[0.2422]]), '루루라라 우리네 인생'), (tensor([[0.1538]]), '결혼생활 그림일기'), (tensor([[0.1366]]), '강남의 기사'), (tensor([[0.2197]]), '리턴 투 플레이어'), (tensor([[0.1377]]), '악당과 악당이 만나면'), (tensor([[0.3014]]), '이별 후 사내 결혼'), (tensor([[0.3514]]), '더블클릭'), (tensor([[0.3917]]), '서브 남주가 파업하면 생기는 일'), (tensor([[0.2469]]), '또다시, 계약 부부'), (tensor([[0.2891]]), 

In [24]:
story_sim_list.sort(reverse=True)

In [28]:
for doc in docs:
    print(doc)